In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
from datetime import datetime
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [3]:
plt.rc('figure',dpi=100);

In [4]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)[['person','label']].head(1960)
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [5]:
def elegir_columnas_random(columnas,n):
    rd.seed(n)
    lista_columnas = columnas.copy()
    cantidad = rd.randint(4, len(lista_columnas))
    columnas_prohibidas = ['person','label','checkout']
    for x in range(cantidad):
        c = rd.choice(lista_columnas)
        if c in columnas_prohibidas:
            continue
        lista_columnas.remove(c)
    return lista_columnas

In [6]:
def grid_search():
    n_estimators = [20,30,40,50,60,70]
    min_samples_split = [10,20,30,40,50]
    n_jobs = [-5,-1]
    maximos = []
    max_score = 0
    for n_est in n_estimators:
        for min_sam in min_samples_split:
            for n_j in n_jobs:
                c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
                dt=c.fit(x_train,y_train)
                score=c.score(x_test,y_test)*100
                if max_score < score:
                    maximos = [n_est,min_sam,n_j]
                    max_score = score
                #print("n_estimators : "+ str(n_est)+"; min_samples_split: "+str(min_sam)+"; n_job: "+str(n_j)+"; SCORE: "+ str(score))
    #print("El mejor score fue: "+str(max_score))
    return maximos

In [7]:
def actualizar_score(score,arbol_score,mejor_arbol_score,cols):
    if score > arbol_score:
        arbol_score = score
    if score > mejor_arbol_score:
        columnas_arbol_mejor_score = cols.copy()
        mejor_arbol_score = score
        sem = x
    return score,arbol_score,mejor_arbol_score,cols

In [8]:
pd.read_csv("data_set.csv", low_memory = False).columns

Index(['person', 'Email', 'Organic', 'Paid', 'Referral', 'Social', 'Unknown',
       'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'promedio dias',
       'promedio hora', 'retornos', 'longitud busqueda', 'coincide',
       'sessions', 'total_time', 'mean_time_by_session', 'max_session_time',
       'mean_events_by_session', 'max_events_by_session',
       'days_since_last_session', 'promedio de TFIdf'],
      dtype='object')

In [9]:
columnas = ['person', 'Email', 'Organic', 'Paid', 'Referral', 'Social', 'Unknown',
       'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'promedio dias',
       'promedio hora', 'retornos', 'longitud busqueda', 'coincide',
       'sessions', 'total_time', 'mean_time_by_session', 'max_session_time',
       'mean_events_by_session', 'max_events_by_session',
       'days_since_last_session', 'promedio de TFIdf'] #Deben ser columnas del csv que se pasa con los features

In [ ]:
RANDOM_SEED = 123
predicciones = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
mejor_arbol_score = 0
columnas_arbol_mejor_score = []
sem = 0
arbol_score = 0
for x in range(200):
    print("Arbol : "+str(x))
    
    #Preparo los dataframe
    training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False).head(1960)#la idea es pasar un set que sean mitad 0 y mitad 1 sus labels
    labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
    data = pd.read_csv("data_set.csv", low_memory = False)
    training_labels = training_labels[['person','label']]
    cols = elegir_columnas_random(columnas,x)
    data = data[cols]
    features = pd.merge(training_labels, data, on='person', how='inner')
    features = features.fillna(0)
    
    #Acomodo los sets de test y entrenamiento
    train, test = train_test_split(features,test_size=0.10,random_state=RANDOM_SEED)
    features=list(features.columns)
    features.remove('person')
    features.remove('label')
    x_train=train[features]
    y_train=train['label']
    x_test=test[features]
    y_test=test['label']
    
    #Entreno arbol con los mejores hiperparametros encontrados
    mejores_hiperparmetros = grid_search()
    n_est= mejores_hiperparmetros[0]
    min_sam = mejores_hiperparmetros[1]
    n_j= mejores_hiperparmetros[2]
    c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
    dt=c.fit(x_train,y_train)
    
    #Veo los resultados
    score=c.score(x_test,y_test)*100
    score,arbol_score,mejor_arbol_score,cols = actualizar_score(score,0,mejor_arbol_score,cols)
    
    #Printeo resultados
    print("Mejor score hasta ahora: "+str(mejor_arbol_score))
    print("Score del arbol: "+str(x)+" es "+str(arbol_score))
    print("Las columnas fueron :" +str(cols))
    print('#--------------------------------------------------------')
    
    #Opcional por si se quiere usar un promedio de todos los resultados para predecir
    prediccion=pd.merge(labels_predict,data,on='person',how='inner')
    prediccion = prediccion.fillna(0)
    x_final=prediccion[features]
    y_final=c.predict(x_final)
    nombre_columna = 'label' +str(x)
    prediccion[nombre_columna]=y_final
    prediccion = prediccion[['person', nombre_columna]]
    predicciones = pd.merge(prediccion,predicciones,on='person',how = 'inner')
    
#Printeo mejor arbol individual    
print('#--------------------------------------------------------')
print("Mejores columnas: "+str(columnas_arbol_mejor_score))
print("Mejor score: "+str(mejor_arbol_score))
print('#--------------------------------------------------------')

In [ ]:
predicciones.to_csv('../modelos/RandomForestResults_ensamble.csv',index=False)